In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 5.5
fig_height = 3.5
fig_format = 'pdf'
fig_dpi = 300

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks':
  os.chdir(r'/Users/alvaroromangomez/Documents/Documentos personales/Formación/Master Bioinformática y Estadística/TFM/tfm_alvaro/Codigo/notebooks')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/3s/vv1d0lmn7g134m4psncn2_q80000gn/T/ipykernel_13047/1023205354.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap.py": 1685228246.3990028, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/importlib/_bootstrap_external.py": 1685228246.4039905, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codecs.py": 1685228243.6488864, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/aliases.py": 1685228245.1487875, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/__init__.py": 1685228245.143839, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/encodings/utf_8.py": 1685228245.728661, "/Users/alvaroromangomez/.pyenv/versions/3.10.10/Library

In [2]:
# PATH TO CUSTOM MODULES
import sys

sys.path.append("../src")

# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from genetic_selection import GeneticSelectionCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# IMPORT CUSTOM MODULES
import utils.statisticsUtils as st

In [3]:
# DIRECTORIES
input_path = "../data/raw/"
input_train_path = "../data/raw/train_data/"
input_test_path = "../data/raw/test_data/"

output_train_path = "../data/processed/train_data/"
output_test_path = "../data/processed/test_data/"

In [4]:
# FILES
molecular_descriptors_training_file = "molecular_descriptors_training.csv"
molecular_descriptors_test_file = "molecular_descriptors_test.csv"

In [5]:
dataset_name = "molecular_descriptors"

In [6]:
# LOAD DATA
molecular_descriptors = pd.read_csv(
    input_train_path + molecular_descriptors_training_file
)
X = molecular_descriptors.drop(columns=["activity"])
Y = molecular_descriptors["activity"]

In [7]:
# REMOVE VARIABLES WITH LOW VARIANCE
molecules_low_var_filter = st.low_variance_filter(X, 0.1)
X_filter = X[molecules_low_var_filter]

In [8]:
# REMOVE VARIABLES WITH HIGH PEARSON CORRELATION
molecules_corr_filter_pearson = st.pearson_corr_filter(X_filter, 0.8, Y)

In [9]:
X_filter = X_filter.drop(columns=molecules_corr_filter_pearson)

In [10]:
# REMOVE VARIABLES WITH HIGH SPEARMAN CORRELATION
molecules_corr_filter_spearman = st.spearman_corr_filter(X_filter, 0.8, Y)

In [11]:
X_filter = X_filter.drop(columns=molecules_corr_filter_spearman)

In [12]:
# SCALE DATA FOR PCA
scaler = MinMaxScaler()
X_filter_scaled = pd.DataFrame(scaler.fit_transform(X_filter), columns=X_filter.columns)

In [13]:
# PCA 95% VARIANCE
pca = PCA(n_components=0.95)
# DATAFRAME
X_pca = pd.DataFrame(pca.fit_transform(X_filter_scaled))

In [14]:
# PLOT PCA
pca.fit(X_filter_scaled)
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")

Text(0, 0.5, 'Cumulative explained variance')

<Figure size 3000x1800 with 1 Axes>

In [15]:
# PLOT TWO FIRST COMPONENTS
plt.figure(figsize=(10, 6))
plt.scatter(X_pca[0], X_pca[1], c=Y, cmap="plasma")
plt.xlabel("First principal component")
plt.ylabel("Second Principal Component")

Text(0, 0.5, 'Second Principal Component')

<Figure size 3000x1800 with 1 Axes>

In [16]:
# PLOT THREE FIRST COMPONENTS 3D
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(X_pca[0], X_pca[1], X_pca[2], c=Y, cmap="plasma")
ax.set_xlabel("First principal component")
ax.set_ylabel("Second Principal Component")
ax.set_zlabel("Third Principal Component")

Text(0.5, 0, 'Third Principal Component')

<Figure size 3000x1800 with 1 Axes>

In [17]:
# GENETIC ALGORITHM FEATURE SELECTION
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [18]:
# TRAIN MODEL
model = RandomForestClassifier(random_state=100)

In [19]:
# SCALED VARIABLES
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [20]:
model.fit(X_scaled, Y)

RandomForestClassifier(random_state=100)

In [21]:
# SELECT FEATURES
selector = GeneticSelectionCV(
    model,
    cv=5,
    verbose=1,
    scoring="roc_auc",
    crossover_proba=0.5,
    mutation_proba=0.2,
    n_generations=50,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.05,
    tournament_size=3,
    n_gen_no_change=20,
    caching=True,
    n_jobs=-1,
)
selector = selector.fit(X_scaled, Y)

Selecting features with genetic algorithm.


gen	nevals	avg                                  	std                               	min                            	max                                  
0  	300   	[   0.573265  128.873333    0.092016]	[  0.018993  74.47839    0.014836]	[ 0.496271  1.        0.      ]	[   0.628441  251.          0.127686]


1  	184   	[   0.58419   136.536667    0.09442 ]	[  0.015855  60.05205    0.013189]	[ 0.52863  9.       0.01618]   	[   0.632849  248.          0.127686]


2  	176   	[   0.590926  124.78        0.097293]	[  0.014292  48.874925   0.011463]	[  0.554949  14.         0.05134 ]	[   0.632849  243.          0.126191]


3  	183   	[   0.597088  110.743333    0.099676]	[  0.014172  41.634891   0.011473]	[  0.542782  14.         0.05134 ]	[   0.636853  235.          0.13567 ]


...

46 	190   	[  0.707132  58.54       0.09736 ]   	[ 0.018435  3.396331  0.008577]   	[  0.645291  49.         0.05713 ]	[  0.728046  77.         0.119248]   


47 	182   	[  0.704882  59.056667   0.097366]   	[ 0.020474  3.740783  0.009494]   	[  0.640918  51.         0.061625]	[  0.728046  76.         0.130338]   


48 	190   	[  0.706111  58.703333   0.096083]   	[ 0.019928  3.390475  0.010232]   	[  0.634319  51.         0.064361]	[  0.728046  72.         0.127129]   


49 	166   	[  0.706648  58.813333   0.094813]   	[ 0.020248  3.640305  0.010795]   	[  0.632669  51.         0.060649]	[  0.728046  75.         0.118928]   


50 	181   	[  0.703603  58.86       0.092685]   	[ 0.020489  3.425454  0.011632]   	[  0.634001  52.         0.056652]	[  0.728046  72.         0.120528]   


In [22]:
# SELECT GENETIC SELECTION VARIABLES
X_genetic_selection = X[X.columns[selector.support_]]

In [23]:
# MIN-MAX SCALER
scaler = MinMaxScaler()
# SCALE FILTER SELECTION DATA
X_filter = pd.DataFrame(scaler.fit_transform(X_filter), columns=X_filter.columns)
# SCALE PCA SELECTION DATA
X_pca = pd.DataFrame(scaler.fit_transform(X_pca), columns=X_pca.columns)
# SCALE GENETIC SELECTION DATA
X_genetic_selection = pd.DataFrame(
    scaler.fit_transform(X_genetic_selection), columns=X_genetic_selection.columns
)

In [24]:
# CONCATENATE X AND Y
## FILTER SELECTION
molecular_descriptors_filter = pd.concat([X_filter, Y], axis=1)
## PCA SELECTION
molecular_descriptors_pca = pd.concat([X_pca, Y], axis=1)
## GENETIC SELECTION
molecular_descriptors_genetic_selection = pd.concat([X_genetic_selection, Y], axis=1)

In [25]:
# WE DO THE SAME WITH THE TEST DATA
molecular_descriptors_test = pd.read_csv(
    input_test_path + molecular_descriptors_test_file
)
## X TEST
X_test = molecular_descriptors_test.drop(columns=["activity"])
## Y TEST
Y_test = molecular_descriptors_test["activity"]

# TEST DATA FOR FILTER SELECTION
X_filter_test = X_test[X_filter.columns]

## MIN-MAX SCALER FOR X TEST
X_filter_test = pd.DataFrame(
    scaler.fit_transform(X_filter_test), columns=X_filter_test.columns
)

## CONCATENATE X AND Y
molecular_descriptors_filter_test = pd.concat([X_filter_test, Y_test], axis=1)

# TEST DATA FOR PCA SELECTION
## PCA TRANSFORMATION DATAFRAME
X_pca_test = pd.DataFrame(pca.transform(X_filter_test))

## MIN-MAX SCALER FOR X TEST
X_pca_test = pd.DataFrame(scaler.fit_transform(X_pca_test), columns=X_pca_test.columns)

## CONCATENATE X AND Y
molecular_descriptors_pca_test = pd.concat([X_pca_test, Y_test], axis=1)

# TEST DATA FOR GENETIC SELECTION
X_genetic_selection_test = X_test[X_genetic_selection.columns]

## MIN-MAX SCALER FOR X TEST
X_genetic_selection_test = pd.DataFrame(
    scaler.fit_transform(X_genetic_selection_test),
    columns=X_genetic_selection_test.columns,
)

## CONCATENATE X AND Y
molecular_descriptors_genetic_selection_test = pd.concat(
    [X_genetic_selection_test, Y_test], axis=1
)

In [26]:
# SAVE DATA FILTER SELECTION
## SAVE TRAIN DATA
molecular_descriptors_filter.to_csv(
    output_train_path + dataset_name + "_filter_training.csv", index=False
)
## SAVE TEST DATA
molecular_descriptors_filter_test.to_csv(
    output_test_path + dataset_name + "_filter_test.csv", index=False
)

# SAVE DATA PCA SELECTION
## SAVE TRAIN DATA
molecular_descriptors_pca.to_csv(
    output_train_path + dataset_name + "_pca_training.csv", index=False
)
## SAVE TEST DATA
molecular_descriptors_pca_test.to_csv(
    output_test_path + dataset_name + "_pca_test.csv", index=False
)

# SAVE DATA GENETIC SELECTION
## SAVE TRAIN DATA
molecular_descriptors_genetic_selection.to_csv(
    output_train_path + dataset_name + "_genetic_selection_training.csv", index=False
)
## SAVE TEST DATA
molecular_descriptors_genetic_selection_test.to_csv(
    output_test_path + dataset_name + "_genetic_selection_test.csv", index=False
)